# PUPPI Tutorial

This Jupyter notebook demonstrates how to use the `puppi` Python package to process a BioID/AP-MS dataset. We will:

1. Load an example intensity file
2. Run feature engineering
3. Train a PU-learning model and estimate FDR
4. Save the output

In [1]:
from puppi.features import feature_engineering
from puppi.train import train_and_score
import pandas as pd

## Load Example Data

In [2]:
intensity_df = pd.read_csv("input_intensity_dataset.tsv", sep="\t")
intensity_df

,Protein,KRT8_1,KRT8_2,KRT8_3,LMNA_1,LMNA_2,LMNA_3,ACTB_1,ACTB_2,ACTB_3,...,MAPRE3_3,EGFP_1,EGFP_2,EGFP_3,Empty_1,Empty_2,Empty_3,NminiTurbo_1,NminiTurbo_2,NminiTurbo_3
0,IGLV3-21;IGLV3-9,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,10349.4,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,IGKV3D-15,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,26850.7,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,IGKV2-28;IGKV2-29;IGKV2-30;IGKV2-40;IGKV2D-26;...,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,18848.6,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,IGKV3-11;IGKV3D-11,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,37883.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,IGHV3-49,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,40300.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,FAM169A,0.0,0.0,0.0,18955.60,25511.70,7100.19,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5316,ZHX2,0.0,0.0,0.0,6059.18,7605.23,2710.10,0.0,0.0,0.0,...,0.00,16249.50,3799.18,2444.10,0.00,0.00,0.00,0.00,0.00,0.00
5317,MORC2,0.0,0.0,0.0,4914.16,6995.20,8531.25,0.0,0.0,0.0,...,0.00,7233.22,0.00,9045.29,0.00,0.00,0.00,0.00,0.00,0.00
5318,IVNS1ABP,39238.4,34255.2,54790.0,15299.60,23808.70,10559.40,32396.1,27509.0,42473.5,...,9761.35,50919.90,48695.70,61052.20,4351.15,9788.32,9907.35,4901.16,7005.24,7117.24


## Run Feature Engineering

You must specify control (substrings identifying control samples).

In [3]:
controls = ['EGFP', 'Empty', 'NminiTurbo']

features_df = feature_engineering(intensity_df, controls)
features_df.head()

,Bait,Prey,log_fold_change,snr,mean_diff,median_diff,replicate_fold_change_sd,bait_cv,bait_control_sd_ratio,zero_or_neg_fc,nonzero_reps,reps_above_ctrl_med,single_rep_flag,composite_score,global_cv
0,ACTB,CCT8,2.188490,3.416967,3.607493e+06,4587256.9,0.186509,0.040917,0.139811,1,3,3,0,16.742191,1.217581
1,ACTB,FLNA,1.721757,2.553110,3.226178e+06,4348742.0,0.365365,0.110771,0.282812,1,3,3,0,15.403182,1.038630
2,ACTB,AHNAK,1.395880,2.040781,2.567820e+06,3867983.0,0.066947,0.025441,0.051919,1,3,3,0,13.012335,0.789551
3,ACTB,ACTA1,3.491085,46.438511,3.079957e+06,2908418.0,2.740437,0.243724,11.318196,1,3,3,0,12.909304,1.610158
4,ACTB,ACTC1,11.813446,1272.496414,1.003520e+05,71529.6,3439.685503,0.955690,1216.112472,1,2,2,0,11.557553,3.511042


## Train PU-learning Model and Estimate FDR

In [4]:
scores_df = train_and_score(features_df, initial_positives=15, initial_negatives=200)

scores_df


Background flagging based on global_cv ≤ 1.0293
Example flagged entries:
     Prey  global_cv     global_cv_flag
2    AAAS   0.609950  likely background
14  ABCC1   1.012973  likely background
18  ABCE1   0.916396  likely background
19  ABCF1   0.759903  likely background
20  ABCF2   0.897372  likely background


,Bait,Prey,log_fold_change,snr,mean_diff,median_diff,replicate_fold_change_sd,bait_cv,bait_control_sd_ratio,zero_or_neg_fc,nonzero_reps,reps_above_ctrl_med,single_rep_flag,composite_score,global_cv,predicted_probability,FDR,global_cv_flag
0,ACTB,A2M,3.489390,22.297028,3489.876889,0.0,178.376227,1.414214,63.065520,1,1,1,1,0.464994,4.795832,0.8729,0.137917,
1,ACTB,A2ML1,3.887981,38.745611,6084.943556,0.0,309.964887,1.414214,109.589137,1,1,1,1,0.636857,4.795832,0.8946,0.116604,
2,ACTB,AAAS,0.179147,2.476831,1655.612222,3542.1,0.122431,0.108135,0.267832,1,3,3,0,0.025320,0.609950,0.6936,0.383857,likely background
3,ACTB,AAGAB,1.095165,1.417723,5745.414444,12210.5,0.491274,0.229958,0.326017,1,3,3,0,0.120357,1.312424,0.8770,0.137917,
4,ACTB,AAK1,1.200044,1.572472,23371.033333,36988.5,0.758363,0.330088,0.519054,1,3,3,0,0.215805,1.082057,0.9695,0.015860,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26591,MAPRE3,ZWINT,-0.005494,0.703972,-40.136667,10633.3,0.067684,0.067942,0.047830,0,3,3,0,0.247697,0.951283,0.2157,1.000000,likely background
26592,MAPRE3,ZYX,-0.701176,0.439522,-26557.394444,42129.4,0.017680,0.028746,0.012634,0,3,3,0,0.183385,1.208568,0.0101,1.000000,
26593,MAPRE3,ZZEF1,-5.536547,0.000000,-4485.592222,0.0,0.000000,0.000000,0.000000,0,0,0,0,-0.343160,1.559795,0.0000,1.000000,
26594,MAPRE3,ZZZ3,-5.536547,0.000000,-5034.066667,0.0,0.000000,0.000000,0.000000,0,0,0,0,-0.345372,1.862704,0.0000,1.000000,


## Save Output

In [5]:
final_df.to_csv("puppi_output.csv", index=False)